In [ ]:
import pandas as pd
import numpy as np
import pickle
import sys
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import Sequential, regularizers 

from keras.layers import Embedding, LSTM, Dense
from keras import losses
from keras.layers import Dense
from keras.layers import Flatten, InputLayer

import time
from keras.callbacks import EarlyStopping, ModelCheckpoint
import torch
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn import metrics
import plotly.express as px

In [ ]:
path_df = "/content/drive/MyDrive/fake_bert/politifact_global_feature.pkl"

In [ ]:
df = pickle.load(open(path_df, "rb"))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['comp_text'], df['target'], test_size= .20, random_state= 111, shuffle=False )

In [ ]:
embed_train, embed_test = pickle.load(open("/content/drive/MyDrive/fake_bert/sbert-embed.pkl", "rb"))

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
cv=5
gs_xgb = XGBClassifier()

param_grid = [
    {'booster':['gbtree', 'dart'], 'n_estimators':[100, 150, 200, 250], 'max_depth':[5, 6, 7]}
]
grid = GridSearchCV(gs_xgb, param_grid, refit=True, cv=cv, verbose=3, scoring='roc_auc')
            

In [ ]:
embed_train.shape

TensorShape([249, 384])

In [ ]:
grid.fit(np.array(embed_train), y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.834 total time=   1.2s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.819 total time=   0.7s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.742 total time=   0.6s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.787 total time=   0.6s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.918 total time=   0.7s
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.834 total time=   0.8s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.816 total time=   0.8s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.752 total time=   1.0s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.792 total time=   0.8s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.920 total time=   0.8s
[CV 1/5] END boost

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid=[{'booster': ['gbtree', 'dart'], 'max_depth': [5, 6, 7],
                          'n_estimators': [100, 150, 200, 250]}],
             scoring='roc_auc', verbose=3)

In [ ]:
predicted_output = grid.predict(np.array(embed_test))

In [ ]:
pickle.dump(grid, open("/content/drive/MyDrive/fake_bert/grid_fea_text_only.pkl", "wb"))

In [ ]:
print(grid.best_params_)

{'booster': 'gbtree', 'max_depth': 7, 'n_estimators': 150}


In [ ]:
print('Accuracy:', accuracy_score(predicted_output, y_test))
print('F1 score:', f1_score(y_test, predicted_output))
print('Recall:', recall_score(y_test, predicted_output))
print('Precision:', precision_score(y_test, predicted_output))
print('ROC_AUC Score:', roc_auc_score(y_test, predicted_output))
print(classification_report(y_test, predicted_output))

Accuracy: 0.7777777777777778
F1 score: 0.7741935483870969
Recall: 0.75
Precision: 0.8
ROC_AUC Score: 0.7782258064516128
              precision    recall  f1-score   support

           0       0.76      0.81      0.78        31
           1       0.80      0.75      0.77        32

    accuracy                           0.78        63
   macro avg       0.78      0.78      0.78        63
weighted avg       0.78      0.78      0.78        63

